In [0]:
srdd = spark.sparkContext.textFile('/FileStore/tables/sharemarket.csv')
srdd.take(5)

Out[35]: ['N,N1,IRFC,BOND 8.00% PA TAX FREE S1,1086,1085,1085,1084.54,1085,3371959.44,3108, ,8,1194,1050',
 'N,N1,JNPT,BOND 6.82% PA TAX FREE S1,1001,1007.1,1015,1007.1,1015,51370,51, ,2,1529.99,1000',
 'N,N1,NHAI,BOND 8.20% PA TAX FREE S1,1091.38,1092,1092,1087,1089.98,3861145.49,3544, ,39,1140,1045',
 'N,N1,NTPC,8.41%S-R-NCD SERIES 1A,1198.99,1135,1135,1135,1135,113500,100, ,1,1275,1022.1',
 'N,N1,RECLTD,TAXFREE SEC NCD TR1 S1,1083.54,1088,1088,1085,1085,108530,100, ,2,1184.8,1041']

In [0]:
srdd2 = srdd.map(lambda x: x.split(','))
head = srdd2.first()
srdd3 = srdd2.filter(lambda x: x!= head)
srdd3.cache()

In [0]:
head = [
    'MARKET','SERIES','SYMBOL','SECURITY','PREV_CL_PR','OPEN_PRICE','HIGH_PRICE','LOW_PRICE','CLOSE_PRICE','NE'
'T_TRDVAL','NET_TRDQTY','CORP_IND','TRADES','HI_52_WK','LO_52_WK'
]

sharedf = srdd3.toDF(head)
sharedf.printSchema()
sharedf.show(5)

root
 |-- MARKET: string (nullable = true)
 |-- SERIES: string (nullable = true)
 |-- SYMBOL: string (nullable = true)
 |-- SECURITY: string (nullable = true)
 |-- PREV_CL_PR: string (nullable = true)
 |-- OPEN_PRICE: string (nullable = true)
 |-- HIGH_PRICE: string (nullable = true)
 |-- LOW_PRICE: string (nullable = true)
 |-- CLOSE_PRICE: string (nullable = true)
 |-- NET_TRDVAL: string (nullable = true)
 |-- NET_TRDQTY: string (nullable = true)
 |-- CORP_IND: string (nullable = true)
 |-- TRADES: string (nullable = true)
 |-- HI_52_WK: string (nullable = true)
 |-- LO_52_WK: string (nullable = true)

+------+------+---------+--------------------+----------+----------+----------+---------+-----------+----------+----------+--------+------+--------+--------+
|MARKET|SERIES|   SYMBOL|            SECURITY|PREV_CL_PR|OPEN_PRICE|HIGH_PRICE|LOW_PRICE|CLOSE_PRICE|NET_TRDVAL|NET_TRDQTY|CORP_IND|TRADES|HI_52_WK|LO_52_WK|
+------+------+---------+--------------------+----------+----------+----

In [0]:
from pyspark.sql.types import *

newDF=[StructField('MARKET',StringType(),True),
       StructField('SERIES',StringType(),True),
       StructField('SYMBOL',StringType(),True),
       StructField('SECURITY',StringType(),True),
       StructField('PREV_CL_PR',FloatType(),True),
       StructField('OPEN_PRICE',FloatType(),True),
       StructField('HIGH_PRICE',FloatType(),True),
       StructField('LOW_PRICE',FloatType(),True),
       StructField('CLOSE_PRICE',FloatType(),True),
       StructField('NET_TRDVAL',FloatType(),True),
       StructField('NET_TRDQTY',FloatType(),True),
       StructField('CORP_IND',StringType(),True),
       StructField('TRADES',FloatType(),True),
       StructField('HI_52_WK',FloatType(),True),
       StructField('LO_52_WK',FloatType(),True)
       ]
finalStruct=StructType(fields=newDF)
df=spark.read.csv('/FileStore/tables/sharemarket.csv',schema=finalStruct)
df.printSchema()
df.show(3)

root
 |-- MARKET: string (nullable = true)
 |-- SERIES: string (nullable = true)
 |-- SYMBOL: string (nullable = true)
 |-- SECURITY: string (nullable = true)
 |-- PREV_CL_PR: float (nullable = true)
 |-- OPEN_PRICE: float (nullable = true)
 |-- HIGH_PRICE: float (nullable = true)
 |-- LOW_PRICE: float (nullable = true)
 |-- CLOSE_PRICE: float (nullable = true)
 |-- NET_TRDVAL: float (nullable = true)
 |-- NET_TRDQTY: float (nullable = true)
 |-- CORP_IND: string (nullable = true)
 |-- TRADES: float (nullable = true)
 |-- HI_52_WK: float (nullable = true)
 |-- LO_52_WK: float (nullable = true)

+------+------+------+--------------------+----------+----------+----------+---------+-----------+----------+----------+--------+------+--------+--------+
|MARKET|SERIES|SYMBOL|            SECURITY|PREV_CL_PR|OPEN_PRICE|HIGH_PRICE|LOW_PRICE|CLOSE_PRICE|NET_TRDVAL|NET_TRDQTY|CORP_IND|TRADES|HI_52_WK|LO_52_WK|
+------+------+------+--------------------+----------+----------+----------+---------+--

In [0]:
# 1.Query to display the number of series present in the data.(using hive)

# 2.Display the series present in the data.(using hive)

# 3.Find the sum of all the prices in the each series. (Using hive)

# 4.Display security,series with highest net trade value (use pyspark)

# 5.Display the series whose sum of all prices greater than the net trade value.(Using pyspark)

# 6.Display the series with highest net trade quantity. (Using pyspark)

# 7. Display the highest and lowest open price(Using sql)

# 8.Query to display the series which have trades more than 80. (Using SQL).

# 9.Display the difference between the net trade value net trade quantity for each series. (Using sql).

In [0]:
df.registerTempTable('sharemarket')

In [0]:
# 1.Query to display the number of series present in the data.

query = '''
select count(distinct(series)) from sharemarket
'''

spark.sql(query).show()


+----------------------+
|count(DISTINCT series)|
+----------------------+
|                    53|
+----------------------+



In [0]:
# 2. Display the series present in the data.

query = '''
select distinct(series) from sharemarket
'''

spark.sql(query).show(5)

+------+
|series|
+------+
|    N3|
|    N2|
|    N5|
|    N4|
|    N6|
+------+
only showing top 5 rows



In [0]:
#  3. Find the sum of all the prices in the each series

query = '''
select series, sum(OPEN_PRICE+HIGH_PRICE+LOW_PRICE+CLOSE_PRICE) totalSUM from sharemarket group by series
'''

spark.sql(query).show()


+------+------------------+
|series|          totalSUM|
+------+------------------+
|    YH|  4098.02001953125|
|    NS|            4129.0|
|    NL|13611.079711914062|
|    NK|  7731.47998046875|
|    Z4|   4124.2001953125|
|    NJ|     17155.1796875|
|    NX|1275.8800048828125|
|    YS|            4530.0|
|    NW|  4356.10009765625|
|    NA|   79628.908203125|
|    Z8|            4151.0|
|    YA|  1113.97998046875|
|    NP| 5172.200012207031|
|    N3| 8348.119918823242|
|    NH|  18790.3603515625|
|    NE| 67285.43798828125|
|    NC|    57412.30859375|
|    NQ|            1260.0|
|    Z1|  5116.10986328125|
|    NR| 4891.440002441406|
+------+------------------+
only showing top 20 rows



In [0]:
# 4. Display security,series with highest net trade value

query = '''
select security, series, max(NET_TRDVAL) highest_net_trade from sharemarket group by security, series
'''

spark.sql(query).show()



+--------------------+------+-----------------+
|            security|series|highest_net_trade|
+--------------------+------+-----------------+
|BOND 8.00% PA TAX...|    N1|        3371959.5|
|SEC RED NCD 8.30%...|    N6|        392325.44|
|SEC RED NCD 9.70%...|    YS|          79125.0|
|7.64% TAX FREE TR...|    N2|        125060.45|
|BOND 0% 2022 TR-3...|    NE|        138086.03|
|BOND8.88% PA TF T...|    NF|         589950.0|
| 9.00% UNSECURED NCD|    N2|         423116.5|
|UNSE RE NCD 0% SR...|    NG|         148400.0|
|BOND 7.03% PA TAX...|    N6|           1341.0|
|BOND 0% 2021 TR-1...|    NA|         91604.08|
|DHANI LOANS & SER...|    N6|          19285.0|
|SERENCD9.00%SRVCI...|    N9|          20400.0|
|TAXFREE SEC NCD T...|    N1|         108530.0|
|8.49% SEC NON-CUM...|    N7|        1993104.1|
|8.79% SEC RED BON...|    N5|         283906.1|
|8.92% SEC RED BON...|    N6|        1400000.0|
|SEC RED NCD 9.60%...|    NX|         12769.72|
|SE RE NCD 8.60% S...|    Y5|         10

In [0]:
# 5. Display the series whose sum of all prices greater than the net trade value.

query = '''
select series, NET_TRDVAL, sum(OPEN_PRICE+HIGH_PRICE+LOW_PRICE+CLOSE_PRICE) highest_net_trade from sharemarket
group by series, NET_TRDVAL
having sum(OPEN_PRICE+HIGH_PRICE+LOW_PRICE+CLOSE_PRICE) > NET_TRDVAL
'''

spark.sql(query).show()


+------+----------+-----------------+
|series|NET_TRDVAL|highest_net_trade|
+------+----------+-----------------+
|    NB|    1843.0|           3686.0|
|    N2|    3666.0|           4888.0|
|    N5|    1912.4|3824.800048828125|
|    N6|    1341.0|           5364.0|
|    N4|   2044.02|  4088.0400390625|
+------+----------+-----------------+



In [0]:
# 6. Display the series with highest net trade quantity

query = '''
select series, NET_TRDQTY from sharemarket
where NET_TRDQTY = (select max(NET_TRDQTY) from sharemarket)
'''

spark.sql(query).show()

+------+----------+
|series|NET_TRDQTY|
+------+----------+
|    N3|  143810.0|
+------+----------+



In [0]:
# 7. Display the highest and lowest open price

query = '''
select max(OPEN_PRICE), min(OPEN_PRICE) from sharemarket
'''

spark.sql(query).show()


+---------------+---------------+
|max(OPEN_PRICE)|min(OPEN_PRICE)|
+---------------+---------------+
|        11450.0|          13.75|
+---------------+---------------+



In [0]:
# 8. Query to display the series which have trades more than 80.

query = '''
select series, TRADES from sharemarket where TRADES > 80
'''

spark.sql(query).show()


+------+------+
|series|TRADES|
+------+------+
|    N2| 107.0|
|    N3| 787.0|
|    N7| 106.0|
+------+------+



In [0]:
# 9. Display the difference between the net trade value net trade quantity for each series.

query = '''
select series, NET_TRDVAL, NET_TRDQTY, NET_TRDVAL - NET_TRDQTY diff from sharemarket
'''

spark.sql(query).show(10)

+------+----------+----------+---------+
|series|NET_TRDVAL|NET_TRDQTY|     diff|
+------+----------+----------+---------+
|    N1| 3371959.5|    3108.0|3368851.5|
|    N1|   51370.0|      51.0|  51319.0|
|    N1| 3861145.5|    3544.0|3857601.5|
|    N1|  113500.0|     100.0| 113400.0|
|    N1|  108530.0|     100.0| 108430.0|
|    N2|  677372.4|   20461.0| 656911.4|
|    N2|    3666.0|       3.0|   3663.0|
|    N2|    4548.0|       4.0|   4544.0|
|    N2|  423116.5|     388.0| 422728.5|
|    N2| 125060.45|     101.0|124959.45|
+------+----------+----------+---------+
only showing top 10 rows

